In [ ]:
#  import sys
# from IPython.display import display

# # Force stdout to flush for real-time prints
# class FlushPrint:
  ###  # @staticmethod
#     def write(msg):
#         sys.__stdout__.write(msg)
#         sys.__stdout__.flush()

# sys.stdout = FlushPrint#


In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# import shutil
# shutil.move('AuthorshipStyleTransferTrain.xlsx', '/content/drive/MyDrive/AraGenEval2025/data/AuthorshipStyleTransferTrain.xlsx')

In [ ]:
# !unzip /content/drive/MyDrive/AraGenEval2025/data/AuthorshipStyleTransferTrain.zip -d /content/drive/MyDrive/AraGenEval2025/data/

In [ ]:
# from torch.utils.data import DataLoader
# from transformers import DataCollatorForSeq2Seq

# # Reuse your tokenizer and model
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# # Simulate DataLoader
# batch_size = 2  # Match your actual training batch size
# loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=data_collator)

# # Inspect first N batches
# num_batches_to_check = 10
# pad_token_id = tokenizer.pad_token_id  # Should be 1 for mBART
# print(f"pad token id {pad_token_id}")

# print(f"\n🔍 Inspecting the first {num_batches_to_check} batches for padding issues:\n")

# for i, batch in enumerate(loader):
#     if i >= num_batches_to_check:
#         break

#     labels = batch["labels"]


#     total_tokens = labels.numel()
#     num_pad_tokens = (labels == pad_token_id).sum().item()
#     num_non_pad_tokens = total_tokens - num_pad_tokens

#     print(f"Batch {i+1}:")
#     print(f"  Total tokens: {total_tokens}")
#     print(f"  Non-pad tokens: {num_non_pad_tokens}")
#     print(f"  Pad tokens: {num_pad_tokens}")
#     print(f"  All padding? {'✅ YES - Loss will be meaningless' if num_non_pad_tokens == 0 else '❌ NO - Valid labels present'}")
#     print("-" * 50)


In [ ]:
# # This cell will attempt to set up the environment for TPU,
# # but is designed to fall back to CUDA/CPU later if TPU is not available.

# # Remove potentially conflicting installations
# !pip uninstall -y torch torch_xla transformers datasets openpyxl psutil

# # Install all primary packages using the Colab TPU index URL.
# # Installing them together like this is crucial for compatibility.
# # We specify the version of transformers etc. if needed, but let pip resolve versions from this index first.
# # If using a specific transformers version is critical, add it here, e.g., transformers==4.30.0
# !pip install -U torch~=2.0.0 torch_xla[tpu] transformers datasets openpyxl psutil \
#     -f https://storage.googleapis.com/libtorch-nightly/cpu/torch-latest.html \
#     --no-cache-dir # Add --no-cache-dir to ensure fresh downloads


# # Verify installations (optional but recommended)
# !pip show torch torch_xla transformers

In [ ]:
# !pip show torch torch_xla transformers

In [ ]:
# import gc
# import torch

# try:
#     import torch_xla
#     print("TPU detected — no need for CUDA memory clear")
# except ImportError:
#     if torch.cuda.is_available():
#         torch.cuda.empty_cache()
# gc.collect()



In [ ]:
# import torch
# print(torch.cuda.is_available())
# print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

# try:
#     import torch_xla.core.xla_model as xm
#     device = xm.xla_device()
#     print(f"Using device: {device} (TPU)")
# except ImportError:
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     print(f"Using device: {device}")

# Fine-tuning mBART for Arabic Authorship Style Transfer (AraGenEval2025)
This notebook mounts Google Drive, loads your dataset, fine-tunes the model, and saves results.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Navigate to project folder
%cd /content/drive/MyDrive/AraGenEval2025
!ls

/content/drive/MyDrive/AraGenEval2025
analysis   EDA		models	     requirements	       submission
config.py  evaluation	prompts      run_baseline_pipeline.py  utils
data	   __init__.py	__pycache__  run_script


In [ ]:
# Install required packages
# %pip install -U transformers datasets openpyxl psutil python-dotenv  # torch_xla
# Install SentencePiece and ensure compatibility
%pip install -U transformers[sentencepiece] sentencepiece datasets openpyxl psutil python-dotenv

In [ ]:
import sys
import os

project_root = os.getcwd()  # Should be /content/drive/MyDrive/AraGenEval2025
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Project root added to sys.path: {project_root}")

Project root added to sys.path: /content/drive/MyDrive/AraGenEval2025


In [ ]:
from config import Config

print(Config.TRAIN_FILE)

data/AuthorshipStyleTransferTrain.xlsx


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import gc

def load_and_sample_dataset(
    file_path,
    sample_mode="all",    # Options: "all", "random", "stratified"
    sample_size=None,     # For "random"/"stratified": float (proportion) or int (absolute count)
    random_state=42
):
    """
    Load dataset from Excel and apply sampling.

    Args:
        file_path (str): Path to dataset file.
        sample_mode (str): "all", "random", or "stratified".
        sample_size (float|int): Proportion (0-1) or count for sampling.
        random_state (int): Random seed for reproducibility.

    Returns:
        pd.DataFrame: Sampled dataset.
    """
    print(f"📂 Loading dataset from: {file_path}")
    df = pd.read_excel(file_path, engine='openpyxl')

    if sample_mode == "all":
        sampled_df = df.copy()
        print(f"✅ Loaded full dataset with {len(sampled_df)} samples.")

    elif sample_mode == "random":
        if sample_size is None:
            raise ValueError("For 'random' mode, SAMPLE_SIZE must be set.")
        sampled_df = df.sample(
            n=sample_size if isinstance(sample_size, int) else int(len(df) * sample_size),
            random_state=random_state
        )
        print(f"✅ Randomly sampled {len(sampled_df)} samples.")

    elif sample_mode == "stratified":
        if sample_size is None or not (0 < sample_size < 1):
            raise ValueError("For 'stratified' mode, SAMPLE_SIZE must be a proportion between 0 and 1.")
        sampled_df, _ = train_test_split(
            df,
            train_size=sample_size,
            stratify=df['author'],
            random_state=random_state
        )
        print(f"✅ Stratified sampled {len(sampled_df)} samples (author distribution preserved).")

    else:
        raise ValueError(f"Invalid sample_mode: {sample_mode}")

    del df
    gc.collect()

    return sampled_df


In [ ]:
from datasets import Dataset

# === CONFIG ===
SAMPLE_MODE = "stratified"  # "all", "random", "stratified"
SAMPLE_SIZE = 0.1           # 10% for stratified or random. Set to None for full dataset.

# === LOAD AND SAMPLE ===
train_df = load_and_sample_dataset(
    Config.TRAIN_FILE,
    # sample_mode=SAMPLE_MODE,
    # sample_size=SAMPLE_SIZE
)

# === ADD REQUIRED COLUMNS ===
train_df['input_text'] = train_df['author'].astype(str) + ' [SEP] ' + train_df['text_in_msa'].astype(str)
train_df['target_text'] = train_df['text_in_author_style'].astype(str)

# === CONVERT TO HF DATASET ===
train_dataset = Dataset.from_pandas(train_df[['input_text', 'target_text']])

print(f"✅ Final dataset ready with {len(train_dataset)} samples.")


📂 Loading dataset from: data/AuthorshipStyleTransferTrain.xlsx
✅ Loaded full dataset with 35122 samples.
✅ Final dataset ready with 35122 samples.


In [ ]:
from config import Config

# print(Config.MODEL_WEIGHTS_FOLDER)

!ls -R {Config.MODEL_WEIGHTS_FOLDER}


models/fine_tuned_model/model_weights/:
checkpoint-1350  checkpoint-1400  checkpoint-1450  checkpoint-1500

models/fine_tuned_model/model_weights/checkpoint-1350:

models/fine_tuned_model/model_weights/checkpoint-1400:
config.json		scheduler.pt		 trainer_state.json
generation_config.json	special_tokens_map.json  training_args.bin
model.safetensors	spiece.model
rng_state.pth		tokenizer_config.json

models/fine_tuned_model/model_weights/checkpoint-1450:
config.json		scheduler.pt		 trainer_state.json
generation_config.json	special_tokens_map.json  training_args.bin
model.safetensors	spiece.model
rng_state.pth		tokenizer_config.json

models/fine_tuned_model/model_weights/checkpoint-1500:
config.json		scheduler.pt		 tokenizer_config.json
generation_config.json	special_tokens_map.json  trainer_state.json
rng_state.pth		spiece.model		 training_args.bin


In [ ]:
# MODEL CONFIGURATION CELL — Modify only this block to switch models
MODEL_CONFIG = {
    "name": "google/mt5-small",  # Options: google/mt5-small, flax-community/arabic-t5-small, CAMeL-Lab/araT5-base, facebook/mbart-large-50-many-to-many-mmt
    "type": "mt5",               # Options: mt5, arabic-t5, mbart
    "input_max_len": 750,
    "target_max_len": 750
}

RESUME_TRAINING = True  # ✅ Change to False to start fresh
CHECKPOINT_DIR = Config.MODEL_WEIGHTS_FOLDER  # Your output_dir already set for checkpoints


In [ ]:
# from transformers import MBart50Tokenizer
# from transformers import T5Tokenizer
from transformers import AutoTokenizer

import matplotlib.pyplot as plt
import seaborn as sns

# tokenizer = MBart50Tokenizer.from_pretrained('facebook/mbart-large-50-many-to-many-mmt')
# tokenizer = T5Tokenizer.from_pretrained("flax-community/arabic-t5-small")
# tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_CONFIG["name"], use_fast=False, legacy=False)

# Only mBART has src_lang/tgt_lang attributes
if MODEL_CONFIG["type"] == "mbart":
    tokenizer.src_lang = 'ar_AR'
    tokenizer.tgt_lang = 'ar_AR'

input_max_len = MODEL_CONFIG.get("input_max_len", 128)
target_max_len = MODEL_CONFIG.get("target_max_len", 128)

print(f"✅ Loaded tokenizer for {MODEL_CONFIG['name']}")


def tokenize_data(example):
    # 1) Tokenize source
    model_inputs = tokenizer(
        example['input_text'],
        max_length=input_max_len,
        truncation=True,
        padding='max_length'
    )

    # 2) Tokenize target
    labels = tokenizer(
        example['target_text'],
        max_length=target_max_len,
        truncation=True,
        padding='max_length'
    )['input_ids']

    # 3) Replace ALL pad_token_id with -100 in labels
    labels = [ (token if token != tokenizer.pad_token_id else -100) for token in labels ]

    model_inputs['labels'] = labels
    return model_inputs


train_dataset = train_dataset.map(
    tokenize_data,
    batched=False,            # Pass one example at a time
    num_proc=4,
    remove_columns=train_dataset.column_names
)
print('Tokenization complete.')

✅ Loaded tokenizer for google/mt5-small


Map (num_proc=4):   0%|          | 0/35122 [00:00<?, ? examples/s]

Tokenization complete.


In [ ]:
import torch
import gc


print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

# Clean up CUDA memory if available and not using TPU
try:
    # Check if CUDA is available AND torch_xla is NOT available (or not in TPU mode)
    # This prevents trying to clear CUDA memory when on TPU, which might cause issues
    using_tpu = False
    try:
        import torch_xla.runtime as xr
        if xr.process_count() > 0: # Check if XLA devices are available
            using_tpu = True
            print("TPU detected — no need for CUDA memory clear")
        else:
             print("torch_xla installed but no TPU devices found.")
    except ImportError:
        print("torch_xla not installed.")


    if torch.cuda.is_available() and not using_tpu:
        torch.cuda.empty_cache()
        print("CUDA memory cleared.")

except Exception as e:
    # Catch potential errors during detection or clearing
    print(f"Error during pre-training memory cleanup: {e}")

gc.collect()

# Safe device detection
try:
    import torch_xla.core.xla_model as xm
    import torch_xla.runtime as xr
    # Check if any XLA devices are actually available
    if xr.process_count() > 0:
         device = xm.xla_device()
         print(f'Using device: {device} (TPU)')
    else:
        # Fallback if torch_xla is installed but no TPU is connected/detected
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f'Using device: {device}')
        if device.type == 'cuda':
            print(f'GPU: {torch.cuda.get_device_name(0)}')
except (ImportError, RuntimeError) as e:
    # Fallback if torch_xla is not installed or encounters a fundamental error
    print(f"Could not initialize TPU. Error: {e}")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')
    if device.type == 'cuda':
        print(f'GPU: {torch.cuda.get_device_name(0)}')

False
No GPU detected
torch_xla not installed.
Could not initialize TPU. Error: No module named 'torch_xla'
Using device: cpu


In [ ]:
if device == 'cuda':
  !nvidia-smi

In [ ]:
!pip install evaluate rouge_score sacrebleu

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.0 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=1b7a8874ad32ab7c3b911037ff41adb29d81ef5fe4088845e3f8d47caccbc4e1
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
import evaluate

chrf_metric = evaluate.load("chrf")
bleu_metric = evaluate.load("bleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 with pad_token_id so labels decode correctly
    labels = [
        [(l if l != -100 else tokenizer.pad_token_id) for l in label]
        for label in labels
    ]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    bleu = bleu_metric.compute(predictions=decoded_preds, references=[[ref] for ref in decoded_labels])
    chrf = chrf_metric.compute(predictions=decoded_preds, references=[[ref] for ref in decoded_labels])

    return {"bleu": bleu["score"], "chrf": chrf["score"]}


In [ ]:
# Model loading

# from transformers import MBartForConditionalGeneration
# from transformers import T5ForConditionalGeneration
from transformers import AutoModelForSeq2SeqLM

from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import psutil
from datetime import datetime
import time

# model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
# model = T5ForConditionalGeneration.from_pretrained("flax-community/arabic-t5-small")
# model     = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")
# print(f"Using lightweight google/mt5-small for debugging")

# model.to(device)
# print(f"✅ Loaded google/mt5-small on {device}")


from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CONFIG["name"])
model.to(device)

print(f"✅ Loaded model {MODEL_CONFIG['name']} on {device}")


# Compute save_steps programmatically
try:
    total_steps = training_args.max_steps
except NameError:
    total_steps = 0  # If not defined yet

# Set save_steps to 5% of max_steps or 100 if max_steps not set or too small
if total_steps and total_steps > 20:
    computed_save_steps = max(1, total_steps // 20)  # 5% of total steps
else:
    computed_save_steps = 50

print(f"💾 Checkpoints will be saved every {computed_save_steps} steps.")


# # Super conservative training settings for 16GB T4 and cpu
# training_args = Seq2SeqTrainingArguments(
#     output_dir=Config.MODEL_WEIGHTS_FOLDER,
#     num_train_epochs=1,
#     per_device_train_batch_size=1,
#     gradient_accumulation_steps=1,
#     learning_rate=1e-4,
#     save_total_limit=1,
#     predict_with_generate=False,
#     fp16=False,
#     logging_steps=1,
#     save_steps=computed_save_steps,  # <-- Programmatic value
#     # max_steps=2,  # Only 2 optimizer steps, stop
#     dataloader_num_workers=0,  # Absolutely no extra workers
#     dataloader_pin_memory=False,
#     gradient_checkpointing=False,
#     optim='adamw_torch',
#     report_to='none',
#     max_grad_norm=1.0
# )


training_args = Seq2SeqTrainingArguments(
    output_dir=Config.MODEL_WEIGHTS_FOLDER,
    # Batch Size & Accumulation
    num_train_epochs=3,               # More epochs for real training
    max_steps=-1,                     # Ensures max_steps is an int—you won't hit the comparison bug                    # Let epochs determine total steps
    per_device_train_batch_size=1,    # Keep batch size low due to long sequences
    gradient_accumulation_steps=4,    # Simulate effective batch size of 4
    learning_rate=5e-5,               # Standard fine-tuning LR
    # Checkpointing & Logging
    save_total_limit=3,               # Keep last 3 checkpoints
    logging_steps=4,                 # Log occasionally
    save_steps=computed_save_steps,   # Will save
    report_to='none',
    # Evaluation
    # evaluate_during_training=False,
    predict_with_generate=False,
    eval_accumulation_steps=1,
    # evaluation_strategy="steps",  # or "epoch"
    # eval_steps=500,               # evaluate every 500 steps
    # predict_with_generate=True,   # required for generation-based metrics
    # evaluation_strategy="steps"
    # optimization
    fp16=False,                       # No FP16 on CPU
    dataloader_num_workers=0,         # Some parallelism for speed
    dataloader_pin_memory=False,
    gradient_checkpointing = True,
    optim='adamw_torch',
    max_grad_norm=1.0,
    auto_find_batch_size=True  # Let trainer avoid memory issues
)



print(f'Memory before training: {psutil.Process().memory_info().rss / 1024 / 1024:.2f} MB')

from transformers import DataCollatorForSeq2Seq

# Use proper data collator for sequence-to-sequence tasks
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=-100  # <— make sure pad tokens in labels become -100
)

from transformers import TrainerCallback, TrainerState, TrainerControl

class PrintLossCallback(TrainerCallback):
    def on_step_end(self, args, state: TrainerState, control: TrainerControl, **kwargs):
        # Grab the last computed loss from state.log_history
        logs = state.log_history[-1]
        if 'loss' in logs:
            print(f"Step {state.global_step} — Loss BEFORE update: {logs['loss']:.4f}")

# Then pass it to your trainer:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    # eval_dataset=val_dataset,
    # compute_metrics=compute_metrics,
    # callbacks=[PrintLossCallback]
)

print(f'Model parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}')

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

✅ Loaded model google/mt5-small on cpu
💾 Checkpoints will be saved every 100 steps.
Memory before training: 3767.60 MB
Model parameters: 300,176,768


In [ ]:
# verify some batches strcuture before training

from torch.utils.data import DataLoader
loader = DataLoader(train_dataset, batch_size=2, collate_fn=data_collator)
batch = next(iter(loader))
# print("Unique labels:", torch.unique(batch['labels']))
print("Count of -100:", (batch['labels']==-100).sum().item())
print("Count of pad_id=1:", (batch['labels']==tokenizer.pad_token_id).sum().item())




Count of -100: 67
Count of pad_id=1: 0


In [ ]:
from datetime import datetime
import time
import os
from config import Config


print(f"📢 Starting training at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

start_time = time.time()

# Ensure logging during training
trainer.args.logging_strategy = "steps"
trainer.args.logging_steps = trainer.args.gradient_accumulation_steps

# Detect checkpoint
last_checkpoint = None
CHECKPOINT_DIR = Config.MODEL_WEIGHTS_FOLDER

if RESUME_TRAINING:
    if os.path.isdir(CHECKPOINT_DIR):
        checkpoints = [d for d in os.listdir(CHECKPOINT_DIR) if d.startswith("checkpoint-")]

        if checkpoints:
            # Sort numerically based on step number
            latest_checkpoint = max(checkpoints, key=lambda x: int(x.split("-")[-1]))
            last_checkpoint = os.path.join(CHECKPOINT_DIR, latest_checkpoint)
            print(f"✅ Found checkpoint at {last_checkpoint}. Will resume training from there.")
        else:
            print("⚠️ No checkpoints found. Starting from scratch.")
    else:
        print("⚠️ Checkpoint directory doesn't exist. Starting from scratch.")
else:
    print("🆕 Starting fresh training (no checkpoint resume).")

# Start training
try:
    if last_checkpoint:
        train_output = trainer.train(resume_from_checkpoint=last_checkpoint)
    else:
        train_output = trainer.train()

    total_time = time.time() - start_time
    print(f"✅ Training completed in {total_time/3600:.2f} hours")

    if train_output.metrics:
        print("\n📊 Final Training Metrics:")
        for key, value in train_output.metrics.items():
            print(f" {key}: {value:.4f}")

except KeyboardInterrupt:
    print("\n⚠️ Training interrupted by user. Saving checkpoint...")
    interrupted_path = os.path.join(CHECKPOINT_DIR, "checkpoint-interrupted")
    trainer.save_model(interrupted_path)
    print(f"💾 Checkpoint saved at {interrupted_path}")


📢 Starting training at 2025-06-25 20:55:24
✅ Found checkpoint at models/fine_tuned_model/model_weights/checkpoint-1300. Will resume training from there.


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].
	save_steps: 100 (from args) != 50 (from trainer_state.json)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
1304,3.280400
1308,3.178500
1312,3.064700
1316,3.231400
1320,3.400100
1324,3.062800
1328,3.141700
1332,3.217300
1336,3.105300
1340,3.268400


Step,Training Loss
1304,3.280400
1308,3.178500
1312,3.064700
1316,3.231400
1320,3.400100
1324,3.062800
1328,3.141700
1332,3.217300
1336,3.105300
1340,3.268400


In [ ]:
# Save final model to Drive
trainer.save_model(Config.MODEL_WEIGHTS_FOLDER)
print('✅ Model saved to Google Drive.')